#### **6. 生成器**
在 Python 语言中，迭代（`for` 循环）是最常用的编程模式之一。程序执行大量的迭代来处理列表、读取文件、查询数据库......Python 最强大的特性之一就是能够以所谓的“生成器函数（generator function）”形式自定义和重新定义迭代。本节将对生成器主题进行介绍。最后，你将学习以一种有趣的方式编写程序来处理实时流数据。

### 01 迭代协议
本节将探究迭代的底层过程。
#### **迭代无处不在 **
许多对象都支持迭代：
```
a = 'hello'
for c in a: # Loop over characters in a
    ...

b = { 'name': 'Dave', 'password':'foo'}
for k in b: # Loop over keys in dictionary
    ...

c = [1,2,3,4]
for i in c: # Loop over items in a list/tuple
    ...

f = open('foo.txt')
for x in f: # Loop over lines in a file
    ...
```
#### **迭代：协议**
考虑以下`for`语句：
```
for x in obj:
    # statements
```
'for'语句的背后发生了什么？
```
_iter = obj.__iter__()        # 获取迭代器对象
while True:
    try:
        x = _iter.__next__()  # 获取下一个元素
    except StopIteration:     # 没有更多元素，则退出循环
        break
```
所有可应用于 `for-loop` 的对象都实现了上述底层迭代协议。

示例：手动迭代一个列表。

In [3]:
x = [1,2,3]
it = x.__iter__()
it
it.__next__()

1

In [4]:
it.__next__()

2

In [5]:
it.__next__()

3

In [6]:
it.__next__()

StopIteration: 

#### **支持迭代**
如果想要将迭代添加到自己的对象中，那么了解迭代非常有用。例如：自定义容器。
```
class Portfolio:
    def __init__(self):
        self.holdings = []

    def __iter__(self):
        return self.holdings.__iter__()
    ...

port = Portfolio()
for s in port:
    ...
```

### 练习
#### **练习 6.1：迭代演示 **
创建以下列表：

In [7]:
a = [1,9,4,25,16]

请手动迭代该列表：先调用`__iter__()`方法获取一个迭代器，然后调用`__next__()`方法获取下一个元素。

In [10]:
i = a.__iter__()
i
print(i.__next__())
print(i.__next__())
print(i.__next__())
print(i.__next__())
print(i.__next__())
print(i.__next__())

1
9
4
25
16


StopIteration: 

内置函数 `next()` 是调用迭代器的 `__next__()` 方法的快捷方式。尝试在一个文件对象上使用 `next()` 方法：

In [15]:
f = open('../Data/portfolio.csv')
f.__iter__()     # 这里返回文件对象自身，因为文件对象实现了迭代协议。

<_io.TextIOWrapper name='../Data/portfolio.csv' mode='r' encoding='cp936'>

In [12]:
next(f)

'name,shares,price\n'

In [13]:
next(f)

'"AA",100,32.20\n'

In [14]:
next(f)

'"IBM",50,91.10\n'

持续调用`next()`，直到文件末尾。方法会抛出一个 `StopIteration` 异常，因为文件对象已经没有更多元素了。

#### **练习 6.2：支持迭代**
有时候，你可能想要使自己的类对象支持迭代——尤其是你的类对象封装了已有的列表或者其它可迭代对象时。请在新的 `portfolio.py` 文件中定义如下类：
```
# portfolio.py

class Portfolio:

    def __init__(self, holdings):
        self._holdings = holdings

    @property
    def total_cost(self):
        return sum([s.cost for s in self._holdings])

    def tabulate_shares(self):
        from collections import Counter
        total_shares = Counter()
        for s in self._holdings:
            total_shares[s.name] += s.shares
        return total_shares
```
Portfolio 类封装了一个列表，同时拥有一些方法，如： `total_cost` property。请修改 `report.py` 文件中的 `read_portfolio()` 函数，以便 `read_portfolio()` 函数能够像下面这样创建 Portfolio 类的实例：
```
# report.py
...

import fileparse
from stock import Stock
from portfolio import Portfolio

def read_portfolio(filename):
    '''
    Read a stock portfolio file into a list of dictionaries with keys
    name, shares, and price.
    '''
    with open(filename) as file:
        portdicts = fileparse.parse_csv(file,
                                        select=['name','shares','price'],
                                        types=[str,int,float])

    portfolio = [ Stock(d['name'], d['shares'], d['price']) for d in portdicts ]
    return Portfolio(portfolio)
...
```
接着运行 `report.py` 程序。你会发现程序运行失败，原因很明显，因为 `Portfolio` 的实例不是可迭代对象。

In [1]:
import report
report.portfolio_report('../Data/portfolio.csv', '../Data/prices.csv')

        名称       股份数量         价格         变化
---------- ---------- ---------- ---------- 
        AA        100       9.22     -22.98
       IBM         50     106.28      15.18
       CAT        150      35.46     -47.98
      MSFT        200      20.89     -30.34
        GE         95      13.48     -26.89
      MSFT         50      20.89     -44.21
       IBM        100     106.28      35.84


可以通过修改 `Portfolio` 类，使 `Portfolio` 类支持迭代来解决此问题：
```
class Portfolio:

    def __init__(self, holdings):
        self._holdings = holdings

    def __iter__(self):
        return self._holdings.__iter__()

    @property
    def total_cost(self):
        return sum([s.shares*s.price for s in self._holdings])

    def tabulate_shares(self):
        from collections import Counter
        total_shares = Counter()
        for s in self._holdings:
            total_shares[s.name] += s.shares
        return total_shares
```
修改完成后， `report.py` 程序应该能够再次正常运行。同时，请修改 `pcost.py` 程序，以便能够像下面这样使用新的 `Portfolio` 对象：
```
# pcost.py

import report

def portfolio_cost(filename):
    '''
    Computes the total cost (shares*price) of a portfolio file
    '''
    portfolio = report.read_portfolio(filename)
    return portfolio.total_cost
...
```
对 `pcost.py` 程序进行测试并确保其能正常工作：
```
>>> import pcost
>>> pcost.portfolio_cost('Data/portfolio.csv')
44671.15
>>>
```

In [1]:
import pcost
pcost.portfolio_cost('../Data/portfolio.csv')

44671.15

#### **练习 6.3：创建一个更合适的容器**
通常，我们创建一个容器类时，不仅希望该类能够迭代，同时也希望该类能够具有一些其它用途。请修改 `Portfolio` 类，使其具有以下这些特殊方法：
```
class Portfolio:
    def __init__(self, holdings):
        self._holdings = holdings

    def __iter__(self):
        return self._holdings.__iter__()

    def __len__(self):
        return len(self._holdings)

    def __getitem__(self, index):
        return self._holdings[index]

    def __contains__(self, name):
        return any([s.name == name for s in self._holdings])

    @property
    def total_cost(self):
        return sum([s.shares*s.price for s in self._holdings])

    def tabulate_shares(self):
        from collections import Counter
        total_shares = Counter()
        for s in self._holdings:
            total_shares[s.name] += s.shares
        return total_shares
```
现在，使用 `Portfolio` 类进行一些实验：

In [1]:
import report
portfolio = report.read_portfolio('../Data/portfolio.csv')
len(portfolio)

7

In [2]:
portfolio[0]

{'name': 'AA', 'shares': 100, 'price': 32.2}

In [3]:
portfolio[1]

{'name': 'IBM', 'shares': 50, 'price': 91.1}

In [4]:
portfolio[0:3]

[{'name': 'AA', 'shares': 100, 'price': 32.2},
 {'name': 'IBM', 'shares': 50, 'price': 91.1},
 {'name': 'CAT', 'shares': 150, 'price': 83.44}]

In [6]:
print('IBM' in portfolio)
# print('AAPL' in portfolio)

False


有关上述代码的一个重要发现——通常，如果一段代码和 Python 的其它代码"类似（speaks the common vocabulary of how other parts of Python normally work）"，那么该代码被认为是 “Pythonic” 的。同理，对于容器对象，其重要组成部分应该包括：支持迭代、可以进行索引、对所包含的元素进行判断，以及其它操作等等。

### 02 自定义迭代
本节探究如何使用生成器函数自定义迭代。
#### **问题**
假设你想要自定义迭代模式。

例如：倒数：
```
for x in countdown(10):
    print(x, end=' ')
10 9 8 7 6 5 4 3 2 1
```
有一个简单方法可以做到这一点。
#### **生成器**
生成器（generator）是定义了迭代的函数：

In [7]:
def countdown(n):
    while n > 0:
        yield n
        n -= 1

示例：

In [8]:
for x in countdown(10):
    print(x, end=' ')

10 9 8 7 6 5 4 3 2 1 

任何使用了 `yield` 语句的函数称为生成器。

生成器函数的行为不同于普通函数。调用生成器函数会创建一个生成器对象（generator object），而不是立即执行函数：

In [9]:
def countdown(n):
    # Added a print statement
    print('Counting down from', n)
    while n > 0:
        yield n
        n -= 1

x = countdown(10)
x

<generator object countdown at 0x0000017255E9E190>

生成器函数只有在`__next__()` 方法被调用时才执行。

In [11]:
x = countdown(10)
x
x.__next__()

Counting down from 10


10

`yield`生成一个值，但是挂起（suspend）函数的执行。生成器函数会在下次调用 `__next__()` 方法时恢复（resume）执行。

In [12]:
x.__next__()

9

In [19]:
x.__next__()

2

当生成器返回最后一个值后，再次迭代将会触发一个错误。

In [20]:
x.__next__()
x.__next__()

StopIteration: 

观察：生成器函数实现的协议与 `for` 语句在列表、元组、字典、文件上使用的底层协议相同。

### 练习
#### **练习 6.4：一个简单的生成器**
如果想要自定义迭代，那么你应该始终考虑生成器函数。生成器函数非常容易编写——创建一个函数，执行所需的迭代逻辑，并使用 `yield` 发送一个值。

例如，创建一个在文件各行中查找匹配子串的生成器：

In [21]:
def filematch(filename, substr):
    with open(filename, 'r') as f:
        for line in f:
            if substr in line:
                yield line
for line in open('../Data/portfolio.csv'):
    print(line, end=' ')

name,shares,price
 "AA",100,32.20
 "IBM",50,91.10
 "CAT",150,83.44
 "MSFT",200,51.23
 "GE",95,40.37
 "MSFT",50,65.10
 "IBM",100,70.44
 

In [22]:
for line in filematch('../Data/portfolio.csv', 'IBM'):
    print(line, end=' ')

"IBM",50,91.10
 "IBM",100,70.44
 

这是一种有趣的思想——你可以在函数中隐藏自定义的处理过程，并将该函数应用于 for 循环。下一个例子探究一种更不寻常的情况。
#### **练习 6.5：监视流数据源**
生成器可应用于监视实时数据源（如：日志文件，股票市场消息）。本部分，我们将对“使用生成器监视实时数据源”这一思想进行探索。首先，请严格遵循以下说明。
`Data/stocksim.py` 用来模仿股市数据，将实时数据不断的写入到 `Data/stocklog.csv` 文件。请打开一个独立的命令行窗口，进入到 `Data/` 目录，然后运行 `stocksim.py` 程序：
```
bash % python3 stocksim.py
```
如果你使用的是 Windows 系统，那么请找到 `stocksim.py` 文件，然后双击该文件运行。然后，让我们先把这个程序放到一边（让它一直在那运行），打开另外一个命令行窗口，查看正在被模拟程序（译注：`stocksim.py`）写入数据的 `Data/stocklog.csv` 文件（译注：如果使用的是 Linux 系统，那么可以进入到 Data 目录下，然后使用 `tail -f stocklog.csv` 命令查看）。你应该可以看到每隔几秒新的文本行被添加到 `Data/stocklog.csv` 文件中。同样，让程序在后台运行——该程序会运行几个小时（对此不用担心）。

`stocksim.py` 程序运行后，让我们编写一个程序来打开 `Data/stocklog.csv` 文件、移动到文件末尾、并查看新的输出。请在 Work 目录下创建 `follow.py` 文件，并把以下代码放入其中：
```
# follow.py
import os
import time

f = open('Data/stocklog.csv')
f.seek(0, os.SEEK_END)   # Move file pointer 0 bytes from end of file

while True:
    line = f.readline()
    if line == '':
        time.sleep(0.1)   # Sleep briefly and retry
        continue
    fields = line.split(',')
    name = fields[0].strip('"')
    price = float(fields[1])
    change = float(fields[4])
    if change < 0:
        print(f'{name:>10s} {price:>10.2f} {change:>10.2f}')
```
运行 `follow.py` 程序，你将会看到实时的股票报价（stock ticker）。 `follow.py` 里的代码类似于 Unix 系统查看日志文件的 `tail -f` 命令。

注意事项：在本示例中，`readline()` 方法的使用与通常从文件中读取行的方式稍微有点不同（通常使用 `for` 循环）。在这种情况下，我们使用 `readline()` 来重复探测文件的末尾，以查看是否添加了新的数据（`readline()` 方法返回新的数据或者空字符串）。

#### **练习 6.6：使用生成器生成数据**
查看练习 6.5 中代码你会发现，代码的第一部分产生了几行数据，而 `while` 循环末尾的语句消费数据。生成器的一个主要特性是你可以将生成数据的代码移动到可重用的函数中。

请修改练习 6.5 的代码，以便通过生成器函数` follow(filename) `执行文件读取。请实现更改以便下面的代码能够工作：
```
>>> for line in follow('stocklog.csv'):
          print(line, end='')

... Should see lines of output produced here ...
```
请修改股票报价代码，使代码看起来像下面这样：
```
if __name__ == '__main__':
    for line in follow('stocklog.csv'):
        fields = line.split(',')
        name = fields[0].strip('"')
        price = float(fields[1])
        change = float(fields[4])
        if change < 0:
            print(f'{name:>10s} {price:>10.2f} {change:>10.2f}')
```

#### **练习 6.7：查看股票投资组合**
请修改 `follow.py` 程序，以便程序能够查看股票数据流，并打印股票投资组合中的那些股票的信息。示例：
```
if __name__ == '__main__':
    import report

    portfolio = report.read_portfolio('Data/portfolio.csv')

    for line in follow('Data/stocklog.csv'):
        fields = line.split(',')
        name = fields[0].strip('"')
        price = float(fields[1])
        change = float(fields[4])
        if name in portfolio:
            print(f'{name:>10s} {price:>10.2f} {change:>10.2f}')
```
注意事项：要想这段代码能够运行， `Portfolio` 类必须支持 `in` 运算符。请参阅 练习 6.3 ，确保 `Portfolio` 类实现了 `__contains__()` 运算符。

#### **讨论**
在这里，你将一个有趣的迭代模式（在文件末尾读取行）移动到函数中。`follow()`函数现在是可以在任何程序中使用的完全通用的实用程序。例如，你可以使用 `follow()` 函数查看服务器日志、调试日志、其它类似的数据源。

### 03 生产者，消费者和管道
生成器在设置各种生产者/消费者问题（produce/consumer problems）和数据流管道（pipeline）中非常有用。本节将对此进行讨论。
#### **生产者/消费者问题**
生成器与各种形式的生产者消费者问题密切相关。
```
# Prodcucer
def follow(f):
    ...
    while True:
        ...
        yield line        # Produces value in `line` below
        ...

# Consumer
for line in follow(f):    # Consumes value from `yield` above
    ...
```
`yield`语句生成给`for`语句消费的值。
#### **生成器管道**
你可以使用生成器的这方面特性来设置进程管道（类似于 Unix 管道（pipe））。

_producer → processing → processing → consumer_

进程管道包括初始的数据生产者、中间的处理阶段、最后的消费者。

_**producer** → processing → processing → consumer_
```
def producer():
    ...
    yield item
    ...
```
通常情况下，生产者是一个生成器，尽管也可以是其它的序列列表。`yield` 将数据输入管道。

_producer → processing → processing → **consumer**_
```
def consumer(s):
    for item in s:
        ...
```
消费者是一个 for 循环，获取数据（译注：items）并对数据执行某些操作。

_producer → **processing → processing** → consumer_
```
def processing(s):
    for item in s:
        ...
        yield newitem
        ...
```
中间的处理阶段同时消费和生产数据。它们可能修改数据流，也可能筛选数据流（丢弃数据）。

_producer → processing → processing → consumer_
```
def producer():
    ...
    yield item          # yields the item that is received by the `processing`
    ...

def processing(s):
    for item in s:      # Comes from the `producer`
        ...
        yield newitem   # yields a new item
        ...

def consumer(s):
    for item in s:      # Comes from the `processing`
        ...
```
设置管道的代码如下：
```
a = producer()
b = processing(a)
c = consumer(b)
```
你会发现数据逐渐地流向不同的函数。

### 练习
对于本练习，`stocksim.py` 程序仍需要在后台运行。并且，你将使用到上一节练习（译注：练习 6.7）编写的 `follow()` 函数。
#### **练习 6.8：创建一个简单的管道**
让我们来看看管道的思想。请创建下面这个函数：
```
>>> def filematch(lines, substr):
        for line in lines:
            if substr in line:
                yield line

>>>
```
`filematch()` 函数除了不再打开文件，几乎与上一节练习的第一个生成器示例完全相同——仅仅对作为参数给出的行序列进行操作。现在，请尝试如下操作：
```
>>> from follow import follow
>>> lines = follow('Data/stocklog.csv')
>>> ibm = filematch(lines, 'IBM')
>>> for line in ibm:
        print(line)

... wait for output ...
```
虽然输出可能需要一定时间才会出现，但是，最后你一定会看到包含 IBM 数据的行。

In [ ]:
def filematch(lines, substr):
    for line in lines:
        if substr in line:
            yield line

from follow import follow
lines = follow('stocklog.csv')
ibm = filematch(lines, 'IBM')
for line in ibm:
    print(line)

#### **练习 6.9：创建一个复杂的管道**
通过执行更多操作来进一步理解管道的思想。

In [ ]:
from follow import follow
import csv
lines = follow('stocklog.csv')
rows = csv.reader(lines)
for row in rows:
    print(row)

这非常有趣。你在这里可以看到，` follow() `函数的输出被传递到 `csv.reader()`函数，并且，我们现在得到了一系列拆分的行。

#### **练习 6.10：创建更多管道组件**
让我们把这样的思想扩展到更大的管道中。首先，创建 `ticker.py` 文件，然后在 `ticker.py` 文件里面创建一个函数，像上面一样读取 CSV 文件：
```
# ticker.py

from follow import follow
import csv

def parse_stock_data(lines):
    rows = csv.reader(lines)
    return rows

if __name__ == '__main__':
    lines = follow('Data/stocklog.csv')
    rows = parse_stock_data(lines)
    for row in rows:
        print(row)
```
接着，创建一个选择特定列的新函数：
```
# ticker.py
...
def select_columns(rows, indices):
    for row in rows:
        yield [row[index] for index in indices]
...
def parse_stock_data(lines):
    rows = csv.reader(lines)
    rows = select_columns(rows, [0, 1, 4])
    return rows
```
再次运行程序，你应该可以看到输出缩小如下：
```
['BA', '98.35', '0.16']
['AA', '39.63', '-0.03']
['XOM', '82.45','-0.23']
['PG', '62.95', '-0.12']
...
```
再接着，创建一个生成器函数以转换数据类型并构建字典。示例：
```
# ticker.py
...

def convert_types(rows, types):
    for row in rows:
        yield [func(val) for func, val in zip(types, row)]

def make_dicts(rows, headers):
    for row in rows:
        yield dict(zip(headers, row))
...
def parse_stock_data(lines):
    rows = csv.reader(lines)
    rows = select_columns(rows, [0, 1, 4])
    rows = convert_types(rows, [str, float, float])
    rows = make_dicts(rows, ['name', 'price', 'change'])
    return rows
...
```
再次运行程序，你应该能够看到像下面这样的字典流：
```
{ 'name':'BA', 'price':98.35, 'change':0.16 }
{ 'name':'AA', 'price':39.63, 'change':-0.03 }
{ 'name':'XOM', 'price':82.45, 'change': -0.23 }
{ 'name':'PG', 'price':62.95, 'change':-0.12 }
...
```

#### **练习 6.11：筛选数据**
创建一个筛选数据的函数。示例：
```
# ticker.py
...

def filter_symbols(rows, names):
    for row in rows:
        if row['name'] in names:
            yield row
```
使用该函数可以筛选出投资组合中的股票：
```
import report
portfolio = report.read_portfolio('Data/portfolio.csv')
rows = parse_stock_data(follow('Data/stocklog.csv'))
rows = filter_symbols(rows, portfolio)
for row in rows:
    print(row)
```
#### **练习 6.12：整合所有的代码**
请在 `ticker.py` 文件中编写函数 `ticker(portfile, logfile, fmt)` ，该函数根据给定的投资组合、日志文件和表格格式创建实时的股票报价器。示例：
```
>>> from ticker import ticker
>>> ticker('Data/portfolio.csv', 'Data/stocklog.csv', 'txt')
      Name      Price     Change
---------- ---------- ----------
        GE      37.14      -0.18
      MSFT      29.96      -0.09
       CAT      78.03      -0.49
        AA      39.34      -0.32
...

>>> ticker('Data/portfolio.csv', 'Data/stocklog.csv', 'csv')
Name,Price,Change
IBM,102.79,-0.28
CAT,78.04,-0.48
AA,39.35,-0.31
CAT,78.05,-0.47
...
```
#### **讨论**
心得体会：你可以创建各种生成器函数，并把它们链接在一起执行涉及数据流的管道处理。另外，你可以创建一个函数，把一系列的管道阶段打包到一个单独的函数中调用（例如 `parse_stock_data()` 函数）。

### 04 有关生成器的更多信息
本节介绍其它与生成器相关的主题，包括生成器表达式（generator expressions）和 itertools 模块。
#### **生成器表达式**
生成器表达式可以理解为列表解析式（list comprehension）的生成器版本：

In [31]:
a = [1,2,3,4]
b = (2*x for x in a)
b
for i in b:
    print(i, end=' ')

2 4 6 8 

生成器表达式与列表列表解析式的区别：
*    不构造列表
*    唯一有用的目的是迭代
*    一旦被消费，无法重复使用
生成器表达式语法：
```
(<expression> for i in s if <conditional>)
```
生成器表达式也可以用作函数参数：

In [32]:
sum(x*x for x in a)

30

生成器表达式可应用于任何迭代：

In [33]:
a = [1,2,3,4]
b = (x*x for x in a)
c = (-x for x in b)
for i in c:
    print(i, end=' ')

-1 -4 -9 -16 

生成器表达式在代码中的一个主要用途是：对序列进行计算，但只使用一次结果。例如，跳过文件中的所有注释。
```
f = open('somefile.txt')
lines = (line for line in f if not line.startswith('#'))
for line in lines:
    ...
f.close()
```
使用生成器，代码运行更快并且占用的内存更少，类似应用于流的过滤器。
#### **为什么使用生成器**
* 许多问题以迭代的形式进行表示会更清晰
    - 对集合中的元素进行遍历，并执行某些操作（如查找、替换，修改等）。
    - 处理管道（processing pipelines）可应用于各种数据处理问题。
* 内存效率更高
    - 只在需要的时候才生成值。
    - 不构造庞大的列表。
    - 可对流数据进行操作。
* 生成器表达式鼓励代码复用
    - 从使用迭代的代码中分离出迭代。
    - 构建迭代函数工具箱，混合搭配（mix-n-match）使用各种工具。
#### **itertools 模块**
`itertools` 是 Python 自带的一个库模块，包含各种函数，旨在帮助开发迭代器/生成器。
```
itertools.chain(s1,s2)
itertools.count(n)
itertools.cycle(s)
itertools.dropwhile(predicate, s)
itertools.groupby(s)
itertools.ifilter(predicate, s)
itertools.imap(function, s1, ... sN)
itertools.repeat(s, n)
itertools.tee(s, ncopies)
itertools.izip(s1, ... , sN)
```
`itertools` 模块中所有的函数都可以迭代地处理数据。并且，这些函数实现了各类迭代模式。

有关生成器的更多信息可以查看 PyCon '08 上的 Generator Tricks for Systems Programmers 教程。

### 练习
在上一节练习中，我们编写代码监视写入日志文件的数据，并将其解析为行序列。

本次练习将以上一节练习中的代码为基础，所以请确保 `Data/stocksim.py` 仍在运行。
#### **练习 6.13：生成器表达式**
生成器表达式是列表解析式的生成器版本。示例：

In [41]:
nums = [1,2,3,4,5]
squares = (x*x for x in nums)
squares

<generator object <genexpr> at 0x0000017256851CB0>

In [42]:
for i in squares:
    print(i, end=' ')

1 4 9 16 25 

与列表解析式不同，生成器表达式只能使用一次。因此，如果你遍历完生成器中的所有元素后，再次对生成器进行遍历，那么你将一无所获。

In [43]:
for i in squares:
    print(i, end=' ')

#### **练习 6.14：函数参数中的生成器表达式**
有时，生成器表达式会作为函数参数使用。虽然这看起来有点怪，但请尝试以下实验：

In [44]:
nums = [1,2,3,4,5]
sum([x*x for x in nums])    # 一个列表推导式
sum(x*x for x in nums)      # 一个生成器表达式

55

在上面的示例中，如果正在操作庞大的列表，那么使用生成器的第二个版本占用的内存会显著减少。

在 `portfolio.py` 文件中，我们执行了一些与列表相关的计算，请尝试将其替换为生成器表达式。

#### **练习 6.15：简化代码**
通常，对于简单的生成器函数，可以使用生成器表达式进行替换。例如，与其这样编写函数：

In [45]:
def filter_symbols(rows, names):
    for row in rows:
        if row['name'] in names:
            yield row

不如使用生成器表达式进行替换：

In [ ]:
rows = (row for row in rows if row['name'] in names)

请修改 `ticker.py` 程序来适当地使用生成器表达式。